In [ ]:
import time

## selenium

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('start-maximized')
# options.add_argument('disable-infobars')
# options.add_argument('--disable-extensions')

### 如果把driver關掉可能會有沒有抓完的情況，因此還是建議打開

In [ ]:
keyword = input("Enter your value: ")
browser = webdriver.Chrome(executable_path=ChromeDriverManager().install(), chrome_options=options)
basic_url = 'https://shopee.tw/search?keyword={keyword}&page={page}'

shop_list = []
sold_list = []
for page in range(2):
    url = basic_url.format(keyword=keyword, page=page)
    print(url)
    
    browser.get(url)

    js = "return action=document.body.scrollHeight"
    height = 0
    new_height = browser.execute_script(js)

    while height < new_height:
        for i in range(height, new_height, 500):
            browser.execute_script('window.scrollTo(0, {})'.format(i))
            time.sleep(3)
        height = new_height
        time.sleep(3)
        new_height = browser.execute_script(js)

    shop_id = browser.find_elements_by_css_selector("a[data-sqe='link']")
    for shop in shop_id:
        shop_href = shop.get_attribute("href")
        shop_id = shop_href.split('.')[-1]
        shop_list.append(shop_id)

    sold_out = browser.find_elements_by_class_name('go5yPW')
    for sold in sold_out:
        if sold.text:
            sold_out = sold.text.split(' ')[-1]
        else:
            sold = '已售出 0'
            sold_out = sold.split(' ')[-1]
        sold_list.append(sold)
            
browser.close()
print('done')

In [ ]:
len(sold_list)

In [ ]:
len(shop_list)

## API

In [4]:
# import logging.config
import logging
import requests
import pandas as pd
from datetime import datetime
# filename = datetime.strftime(datetime.now(), '%Y%m%d_%H%M%S') + '_objective3.log'
# log_path = 'logs/' + filename 
log_path = 'logs/shopee_crawler.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s : %(message)s',
    filename=log_path)

# logging.config.fileConfig(fname='log.conf', defaults={'logfile': filename})
# logger = logging.getLogger('cathy3')
# logger.info('Start objective 3')

# input keyword then search keyword
keyword = (input('enter your keyword:'))
url = ('https://shopee.tw/api/v2/search_items/?by=relevancy&keyword={keyword}' \
       '&limit=50&newest=0&order=desc&page_type={page}&version=2')

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'X-Requested-With': 'XMLHttpRequest',  
}    

shop_list = []
sold_list = []

try:
    # limit 300 products in 6 pages
    for page in range(1, 7):
        single_page = url.format(keyword=keyword, page=page)
        parsing = requests.get(single_page, headers=headers)
        data = parsing.json()
        items_list = data['items']
        
        # if items_list is true then get shop_id & sold_out into list    
        if items_list:
            logging.info('products exist have {count} records'.format(count=len(items_list)))
            for item in items_list:
                shop_list.append(item['shopid'])
                sold_list.append(item['historical_sold'])
            if len(items_list) == int(50):
                pass
            else:
                break
        else:
            break
    logging.info('{count} rows of shop_list'.format(count=len(shop_list)))
    logging.info('{count} rows of sold_list'.format(count=len(sold_list)))
                
except requests.exceptions.HTTPError as error_http:
    logging.exception(error_http)
except requests.exceptions.ConnectionError as error_connect:
    logging.exception(error_connect)
except requests.exceptions.Timeout as error_time:
    logging.exception(error_time)
except requests.exceptions.RequestException as error:
    logging.exception(error)

try:
    # turn data_list to dataframe 
    record_df = pd.DataFrame(columns=['shop_id', 'sold_out'])
    record_df['shop_id'] = shop_list
    record_df['sold_out'] = sold_list
#     logger.info('turn to dataframe')
    # dataframe groupby 'shop_id'
    group_df = record_df.groupby(['shop_id']).agg({'sold_out':sum})
    group_record_df = group_df.reset_index()
    logging.info('complete data groupby {data}'.format(data=group_record_df))
    # dataframe sort by 'sold_out'
    sort_record_df = group_record_df.sort_values(by='sold_out', ascending=False)
    all_record_df = sort_record_df.reset_index(drop=True)
    logging.info('complete data sorting by shop_id {data}'.format(data=all_record_df))
    logging.info('Success groupby & sorting in dataframe')
    
except FutureWarning as warning:
    logging.warning(warning)
except Exception as inst:
    logging.exception(inst)
    
# get 50 data 
if len(all_record_df) > 50 :
    counts = 50
else:
    counts = len(all_record_df)
logging.info('length of dataframe:{count}'.format(count=counts))

# output as Stipulate format
input_word = 'The number {number} shop is {shop_id} with sales amount {sold_out}'
for count in range(counts):
    shop_id = (all_record_df['shop_id'][count])
    sold_out = (all_record_df['sold_out'][count])
    output = input_word.format(number=count+1, shop_id=shop_id, sold_out=sold_out)
    logging.info(output)

logging.info('Finish objective 3')    


enter your keyword:milk
